## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-09-06-07-58-37.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['August']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    934
Name: count, dtype: int64
-------
$/SQUARE FEET
False    928
True       6
Name: count, dtype: int64
-------
YEAR BUILT
False    934
Name: count, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [8]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
245,PAST SALE,August-17-2023,Condo/Co-op,2160 Bay Dr Unit 2-8,Miami Beach,FL,33141.0,210.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2160-Bay...,MARMLS,A11307716,N,Y,25.852043,-80.144581
383,PAST SALE,August-15-2023,Condo/Co-op,842 Meridian Ave Unit 3E,Miami Beach,FL,33139.0,225.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/842-Meri...,MARMLS,A11413156,N,Y,25.778981,-80.136541
673,PAST SALE,August-3-2023,Condo/Co-op,1775 Washington Ave Unit 9F,Miami Beach,FL,33139.0,550.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1775-Was...,MARMLS,A11376729,N,Y,25.793558,-80.131649
777,PAST SALE,August-15-2023,Condo/Co-op,400 Kings Point Dr #917,Sunny Isles Beach,FL,33160.0,27500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/40...,MARMLS,A11366753,N,Y,25.923572,-80.127767
458,PAST SALE,August-30-2023,Condo/Co-op,18707 NE 2nd Ave #919,Miami,FL,33179.0,42000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18707-NE-2nd-A...,Beaches MLS,F10388654,N,Y,25.947076,-80.196878
460,PAST SALE,August-4-2023,Condo/Co-op,15600 NE 6th Ave Unit 12A,Miami,FL,33162.0,91750.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15600-NE-6th-A...,Beaches MLS,F10379170,N,Y,25.917779,-80.187985
131,PAST SALE,August-8-2023,Condo/Co-op,19201 Collins Ave #527,Sunny Isles Beach,FL,33160.0,115000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/19...,MARMLS,A11406388,N,Y,25.954318,-80.119900
9,PAST SALE,August-28-2023,Condo/Co-op,1020 Euclid Ave #5,Miami Beach,FL,33139.0,120000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1020-Euc...,MARMLS,A11436780,N,Y,25.781056,-80.135453
849,PAST SALE,August-4-2023,Condo/Co-op,1351 NE Miami Gardens Dr Ph 1E,Miami,FL,33179.0,120000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1351-NE-Miami-...,MARMLS,A11329011,N,Y,25.945827,-80.174810
31,PAST SALE,August-9-2023,Condo/Co-op,1855 W 62nd St Unit E111,Hialeah,FL,33012.0,133000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/1855-W-62nd-...,MARMLS,A11419725,N,Y,25.879747,-80.320688


In [22]:
print(df_filtered['URL'].iloc[777])

https://www.redfin.com/FL/Sunny-Isles-Beach/400-Kings-Point-Dr-33160/unit-917/home/43255170


In [17]:
# Correct the prices, if needed
df_filtered.at[245,'PRICE']=(210000)
df_filtered.at[383,'PRICE']=(225000)
df_filtered.at[673,'PRICE']=(550000)
df_filtered.at[777,'PRICE']=(275000)

In [18]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [23]:
# # Corrections, if needed
df_filtered.at[245,'$/SQUARE FEET']=(210000/480)
df_filtered.at[383,'$/SQUARE FEET']=(225000/460)
df_filtered.at[673,'$/SQUARE FEET']=(550000/960)
df_filtered.at[777,'$/SQUARE FEET']=(275000/697)

In [24]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
458,42000.0,18707 NE 2nd Ave #919,Miami,103.0
712,150000.0,1780 NE 191st St Unit 502-2,Miami,114.0
759,142500.0,1680 NE 191st St Unit 205-2,Miami,138.0
427,199000.0,1690 NE 191st St Unit 309-1,Miami,139.0
460,91750.0,15600 NE 6th Ave Unit 12A,Miami,148.0
849,120000.0,1351 NE Miami Gardens Dr Ph 1E,Miami,150.0
914,299500.0,1800 NE 114th St #402,Miami,154.0
744,150000.0,671 NE 195th St Unit 417E,Miami,156.0
240,225000.0,511 Ives Dairy Rd Unit 405-6,Miami,158.0
53,170000.0,14820 Naranja Lakes Blvd Unit D1G,Homestead,158.0


In [70]:
print(df_filtered.URL.iloc[8])

https://www.redfin.com/FL/North-Miami/13480-NE-6th-Ave-33161/unit-303/home/42879152


In [71]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [25]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [26]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [28]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [29]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [30]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"August 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [31]:
m.save('index.html')

## Summary Info

In [32]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [33]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [34]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [35]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/18707-NE-2nd-Ave-33179/unit-919/home/43009578


In [36]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
GROVE AT GRAND BAY, 2675 S Bayshore Dr Unit LPHS Miami | Price $19,250,000 | $2,024 psf | Year built: 2016
Least Expensive
Star Lakes Estates, 18707 NE 2nd Ave #919 Miami | Price $42,000 | $103 psf | Year built: 1969
Highest Price Per Square Foot
SURF CLUB CONDO, 9001 Collins Ave Unit S-207 Surfside | Price $18,500,000 | $4,229 psf | Year built: 2017
Lowest Price Per Square Foot
Star Lakes Estates, 18707 NE 2nd Ave #919 Miami | Price $42,000 | $103 psf | Year built: 1969
Newest
REALTY SECURITIES CORPPB, 2859 SW 37th Ct #2861 Miami | Price $1,175,000 | $557 psf | Year built: 2023
Oldest
CASA GRANDE CONDO, 834 Ocean Dr #501 Miami Beach | Price $850,000 | $639 psf | Year built: 1923


## Time on Market Calculator

In [84]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [93]:
from datetime import datetime, timedelta

date1 = datetime(2023, 1, 24) ## List (Earlier) date
date2 = datetime(2023, 7, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

178


## Map URL Snagger

In [86]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [87]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_july_2023_copy


## Get Summary Data

In [37]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 934
--------
Total sale price: $700,109,037
--------
Median sale price: $410,000
--------
Max sale price: $19,250,000
--------
Min sale price: $42,000
------------------------------------------------
PSF INFO
Max price per square foot: $4,229
--------
Min price per square foot: $103
--------
Median price per square foot: $430
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1923.0
----------
Average building age: 1988.237687366167
